# Introducción a los volúmenes

Los volúmenes permiten almacenar data persistente del contenedor

Existen 3 tipos de volúmenes:

- <b>Host</b>: Son volúmenes que se almacenan en nuestro LocalHost y que viven dentro de una carpeta que definimos en nuestro file system.
- <b>Anonymus</b>: Son los que no definimos y Docker automaticamente genera una carpeta Random y almacena la información.
- <b>Named Volumes</b>: Son volúmenes que nosotros creamos, no son carpetas nuestros sino que son administradas por Docker pero que a diferencia de los anoniymus si tienen un Nombre y son manejados netamente por Docker.

# 1. ¿ Por qué son importantes los volúmenes? 

Levantamos un contenedor de MySQL

- docker run -d -p 3306:3306  --name my-db -e "MYSQL_ROOT_PASSWORD=12345678" -e "MYSQL_DATABASE=docker-db" -e "MYSQL_USER=docker-user" -e "MYSQL_PASSWORD=87654321" mysql:5.7

Luego nos conectamos de manera local 

- mysql -u root -h 127.0.0.1 -p12345678

# 2. Volúmenes HOST

Donde guarda la información mysql, R//: en el directorio /var/lib/mysql

Creamos una carpeta para guardar el contenido del contenedor: ejemplo: sudo mkdir mysql en el directorio opt, entonces la información quedaría guardada en la ruta: opt/mysql

Creamos un contenedor, definiendo el volumen:

- docker run -d --name <"CONTAINER_NAME"> -p <"LOCAL_PORT">:<"CONTAINER_PORT"> -e "MYSQL_ROOT_PASSWORD=<"PASS">" -v <"LOCAL_PATH">:<"CONTAINER_PATH"> <"IMAGE_NAME">:<"TAG_NAME">

Ejemplo:

- docker run -d --name db -p 3306:3306 -e "MYSQL_ROOT_PASSWORD=123456789" -v /opt/mysql:/var/lib/mysql mysql:5.7

# 3. Volúmenes anónimos

Solo indicamos al momento de crear el contenedor el path de archivos que queremos que se conserven dentro del contenedor, ejemplo:

- docker run -d --name <"CONTAINER_NAME"> -v <"CONTAINER_PATH"> -e "MYSQL_ROOT_PASSWORD=<"PASS">" -e "MYSQL_DATABASE=<"docker_databases">" <"IMAGE_NAME">:<"TAG">


Ejemplo:

   - docker run -d --name mysql -v /var/lib/mysql -p 3306:3306 -e "MYSQL_ROOT_PASSWORD=12345678" -e "MYSQL_DATABASE=docker-db" mysql:5.7
   
Esto creara al azar el volumen 

Para ver el document root de docker usamos el comando:

- docker info | grep -i root

##### Resultado

 <b>Docker Root Dir: /var/lib/docker</b>

Luego ingresamos a dicha ruta como sudo

1. sudo su
2. cd /var/lib/docker

A continuación se lista todas las carpetas que emplea docker, ingresamos al apartado de volumes

![](images/list_docker_documents.png)

Luego de corre el container, docker creara una carpeta con nombre aleatorio para guardar la información de mysql. 

al explorar los carpetas dentro de volumes encontramos

![](images/volumenes.png)


#### NOTA: No se recomienda ya que son nombres dificiles de comprender

# 4. VOLUMENES dentro de un DOCKERFILE


Listar volumenes:

- docker volume ls

Creamo una imagen con el Dockerfile

FROM centos

VOLUME /opt/

Creamos la images:

- docker build -t test-vol -f Dockerfile_vol .

Levantamos un contenedor:

- docker run -dti--name test test-vol


Luego miramos los volumnes y tenemos una carpeta llamada:

a7868f0cfb334b1ac4bd03e25c21584e0e439ba9b50bff7e3e8e012e78f1ee01

Pero ¿Commo accedemos a dicho volumne fuera del contenedor?

1. buscamos el docker file directory con el comando docker info | grep -i root (Docker Root Dir: /var/lib/docker)
2. Accedemos a la ruta especificada con SUDO: sudo su y luego cd /var/lib/docker/<b>volumes</b>
3. luego ingresamos a la carpeta listada por el volumen y dentro de <b>_data</b>
4. dentro de esta carpeta se encuentra los archivo que creamos 


### OJO : al momento de eliminar los contenedores no usar el parametro -fv  solo usar f ya que la v elimina los volúmenes.

# 5. Named Volumes (Volúmenes nombrados)

Es la union entre un volúmen de host y uno anonimo.

Se usa el comando:

- docker volume create <"VOLUME_NAME">

- docker volume ls 

### ELIMINAR UN VOLUMEN

- docker volume rm <"VOLUME_NAME">

### Asignar volumen a un container

- docker run -d --name <"CONTAINER_NAME"> -v <"VOLUME_NAME">:<"CONTAINER_PATH" ...... 

Ejemplo:

- docker run -d --name mysql -v my-vol:/var/lib/mysql -p 3306:3306 -e "MYSQL_ROOT_PASSWORD=12345678" -e "MYSQL_DATABASE=docker-db" mysql:5.7


# 6. Dangling Volumes

Son volúmenes que ningun contenedor esta utilizando por lo tanto son Basura.

Comando para eliminarlos todos es:

- docker volume ls -f dangling=true -q | xargs docker volume rm

# 7. Persistiendo data de Mongo

- Creo una carpeta donde deseo almacenar la data de mongo en mi local

Luego creo el container asignando el PATH del volúmen.

Ejemplo:

- docker run -d -p 27019:27017 -v /opt/mongo:/data/db mongo

### Ejecutar un comando sin ingresar al container

- docker exec <"CONTAINER_NAME"> bash -c "<"COMAND">"

# 8. Comparte Volúmenes entre uno o más contenedores.

Simplemente se juega con la ruta de los volumenes al momento de crearlos 